<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Working With Time Series Data in FBProphet

_By Bryce Peake (Washington DC) and Steven Longstreet (Washington DC)_

### Learning Objectives
 
**After this lesson, you will be able to:**
- Identify time series data.
- Explain the challenges of working with time series data.
- Use the `datetime` library to represent dates as objects.
- Preprocess time series data with Pandas
- Create and visualize a Time Series model using FBProphet
- Evaluate a Time Series model

<h2><a id="A">What is a Time Series?</a></h2>

A **time series** is a series of data points that's indexed (or listed, or graphed) in time order. Most commonly, a time series is a sequence that's taken at successive equally spaced points in time. Time series are often represented as a set of observations that have a time-bound relation, which is represented as an index.

Time series are commonly found in sales, analysis, stock market trends, economic phenomena, and social science problems.

These data sets are often investigated to evaluate the long-term trends, forecast the future, or perform some other form of analysis.

### Let's take a look at some Apple stock data to get a feel for what time series data look like.

In [ ]:
import pandas as pd
from datetime import timedelta
%matplotlib inline

apple_stock = pd.read_csv("./data/aapl.csv")
apple_stock.head()

<h2><a id="B">The DateTime library</a></h2>
As time is important to time series data, we will need to interpret these data in the ways that humans interpret them (which is many ways). 

Python's `DateTime` library is great for dealing with time-related data, and Pandas has incorporated this library into its own `datetime` series and objects.

In this lesson, we'll review these data types and learn a little more about each of them:

* `datetime` objects.
* `datetime` series.
* Timestamps.
* `timedelta()`.

### `datetime` Objects
Below, we'll load in the `DateTime` library, which we can use to create a `datetime` object by entering in the different components of the date as arguments.

In [ ]:
apple_stock.info()

In [ ]:
# The datetime library is something you should already have from Anaconda.
from datetime import datetime

In [ ]:
# Let's just set a random datetime — not the end of the world or anything.
lesson_date = datetime(2012, 12, 21, 12, 21, 12, 844089)

In [ ]:
type(lesson_date)

In [ ]:
#The components of the date are accessible via the object's attributes.
print("Micro-Second", lesson_date.microsecond)
print("Second", lesson_date.second)
print("Minute", lesson_date.minute)
print("Hour", lesson_date.hour)
print("Day", lesson_date.day)
print("Month",lesson_date.month)
print("Year", lesson_date.year)

### `timedelta()`
Suppose we want to add time to or subtract time from a date. Maybe we're using time as an index and want to get everything that happened a week before a specific observation.

We can use a `timedelta` object to shift a `datetime` object. Here's an example:

In [ ]:
# Import timedelta() from the DateTime library.
from datetime import timedelta

# Timedeltas represent time as an amount rather than as a fixed position.
offset = timedelta(days=1, seconds=20)

# The timedelta() has attributes that allow us to extract values from it.
print('offset days', offset.days)
print('offset seconds', offset.seconds)
print('offset microseconds', offset.microseconds)

`datetime`'s `.now()` function will give you the `datetime` object of this very moment.

In [ ]:
now = datetime.now()
print("Like Right Now: ", now)

The current time is particularly useful when using `timedelta()`.

In [ ]:
print("Future: ", now + offset)
print("Past: ", now - offset)

*Note: The largest value a `timedelta()` can hold is days. For instance, you can't say you want your offset to be two years, 44 days, and 12 hours; you have to convert those years to days.*

You can read more about the `timedelta()` category [here](https://docs.python.org/2/library/datetime.html).

## Preprocessing Time Series Data with Pandas

In [ ]:
#Overwrite the original `Date` column with one that's been converted to a `datetime` series.
apple_stock['Date'] = pd.to_datetime(apple_stock.Date)
apple_stock.dtypes

### The `.dt` Attribute
Pandas' `datetime` columns have a `.dt` attribute that allows you to access attributes that are specific to dates. For example:

    aapl.Date.dt.day
    aapl.Date.dt.month
    aapl.Date.dt.year
    aapl.Date.dt.weekday_name

Check out the Pandas `.dt` [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.dt.html) for more information.

In [ ]:
apple_stock.Date.dt.year

In [ ]:
apple_stock.Date.dt.dayofyear.head()

### Timestamps
Timestamps are useful objects for comparisons. You can create a timestamp object using the `pd.to_datetime()` function and a string specifying the date. These objects are especially helpful when you need to perform logical filtering with dates.

In [ ]:
ts = pd.to_datetime('1/1/2017')
ts


The main difference between a `datetime` object and a `timestamp` is that timestamps can be used as comparisons.



In [ ]:
apple_stock.loc[apple_stock.Date >= ts, :].head()

### Filter by date with Pandas
It is easy to filter by date using Pandas. Let's create a subset of data containing only the stock prices from 2017. We can specify the index as a string constant. 

In [ ]:
#first, set the date (as a datetime obj) as the index
apple_stock.set_index('Date', inplace=True)

In [ ]:
#we can ask for only that data from December 2016
apple_stock['2016-11']

There are a few things to note about indexing with time series. Unlike numeric indexing, the end index will be included. If you want to index with a range, the time indices must be sorted first.  

> **Recap:** The steps for preprocessing time series data are to:
* Convert time data to a `datetime` object.
* Set `datetime` to index the DataFrame.

## Practice
Let's use the UFO data set to build a timeseries

In [ ]:
# Create a `datetime` object representing today's date.
datetime.now().second

In [ ]:
# Load the UFO data set from the internet.

#go to this URL https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv, 
#use your mouse to copy all of the data

ufo = pd.read_clipboard(sep = ',')

In [ ]:
ufo.head()

In [ ]:
# Convert the Time column to a datetime object.
ufo["Time"] = pd.to_datetime(ufo.Time)

In [ ]:
# Set the `Time` column to the index of the dataframe.
ufo.set_index("Time", inplace = True)

In [ ]:
# Create a `timestamp` object for the date January 1, 1999.
timestamp = pd.to_datetime('1/1/1999')

In [ ]:
ufo.shape

In [ ]:
# Create a subset of entries with a date above or equal to January 1, 1999 using a timestamp object.
recent_ufos = ufo.loc[ufo.index >= timestamp, :]
recent_ufos.shape

## Aggregating Time Data
If we want to investigate trends over time in sales, as always, we'll start by computing simple aggregates. We want to know: What were the mean and median sales in each month and year?

We can use `data.resample` on the whole data set and provide:
    - A parameter for the level on which to roll up to: `'D'` for day, `'W'` for week, `'M'` for month, `'A'` for year.
    - The aggregation method to perform: `mean()`, `median()`, `sum()`, etc.

In [ ]:
#import the sales data
data = pd.read_csv('data/rossmann.csv', skipinitialspace=True, low_memory=False)
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')

In [ ]:
data[['Sales']].resample('M').mean()

In [ ]:
data[['Sales']].resample('M').std()

## Rolling Statistics
With time series, we can "roll" statistics across time. For example, the rolling mean is the mean of a moving window across time periods. Pandas offers a variety of functionalities for creating rolling statistics, which we'll only scratch the surface of here.

E.g., to understand holidays sales, we don't want to compare sales data in late December with the entire month but instead with a few days immediately surrounding it. We can do this using rolling averages.

The syntax for these can be a little tricky at first. We'll be using a `rolling()` function with a statistical function chained to it. Let's dive into more detail.

### Parameters for `rolling()` Functions

`rolling().mean()` (as well as `rolling().median())` can take the following parameters:

* The first indicates the time series to aggregate.
* `window` indicates the number of periods to include in the average.
* `center` indicates whether the window should be centered on the date or use data prior to that date.



#### Calculate the rolling daily sum over all stores.
Use the `.resample()` function to calculate the daily total over all of the stores.

In [ ]:
daily_store_sales = data[['Sales']].resample('D').sum()
daily_store_sales

Use the `.rolling()` function to calculate the rolling average over a three-day period.

In [ ]:
daily_store_sales.rolling(window=3, center=False).mean().head(10)

In [ ]:
#We can use our index filtering to just look at 2015
daily_store_sales.rolling( window=7, center=True).mean()['2015'].head()

Instead of plotting the full time series, we can plot the rolling mean instead, which smooths random changes in sales and removes outliers, helping us identify larger trends.

In [ ]:
daily_store_sales.rolling(window=30, center=True).mean().plot()

### The Expanding Mean
The expanding mean simply uses all of the data points up to the current time to calculate the mean, as opposed to a moving window.

#### Calculate and plot the expanding mean below. Resample by quarter.

In [ ]:
rolling_mean = data.Sales.resample('Q').sum().rolling(window=1, center=False).mean()
expanding_mean = data.Sales.resample('Q').sum().expanding().mean()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [16,9]
fig, ax = plt.subplots()
rolling_mean.plot(legend = True)
expanding_mean.plot(legend = True)
ax.legend(['Rolling Mean', 'Expanding Mean'])

### Exponentially Weighted Windows
Exponentially weighted windows are one of the most common and effective ways of averaging out noise in time series data. The averaging is done with an "exponential decay" on the contribution of prior means, decreasing the contribution of time points that are further in the past.

The (adjusted) exponentially weighted mean for time, $t$, is defined as:

<a id="-xt--fracxt-----alphaxt------alphaxt--------alphatx------alpha-----alpha-------alphat-"></a>
### $$ x_t = \frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2x_{t-1} + ... + (1 - \alpha)^{t}x_0} {1 + (1 - \alpha) + (1 - \alpha)^2 + ... + (1 - \alpha)^{t}} $$

> **Note:** Review Pandas' [documentation](http://pandas.pydata.org/pandas-docs/stable/computation.html#exponentially-weighted-windows) for more information.

**Calculate and plot the exponentially weighted sum along with the rolling sum. What's the difference?**

For example: `.resample('Q').sum().ewm(span=10).mean()`.



In [ ]:
rolling_mean = data.Sales.resample('Q').sum().rolling(window=2, center=True).mean()
exp_weighted_mean = data.Sales.resample('Q').sum().ewm(span=10).mean()

In [ ]:
plt.rcParams["figure.figsize"] = [16,9]
fig, ax = plt.subplots()
rolling_mean.plot(legend = True)
exp_weighted_mean.plot(legend = True)
ax.legend(['Rolling Mean', 'Exponentially Weighted Mean'])

*Note that rolling doesn't understand if you are missing periods (e.g., if you roll over three days and your data are missing weekends, it'll roll Fri/Sat/Sun), so you need to resample first if you care about that.*

## Shifting and Lagging Time Series Data
Another common operation on time series data is shifting or lagging values backward and forward in time. This can help us calculate the percentage of change from sample to sample. Pandas has a `.shift()` method for shifting the data in a DataFrame.

Let's take a look at the Rossman data when we apply lagged features. 

In [ ]:
shifted_forward = data.shift(1)
shifted_forward.head()

*Notice that the first row now contains NaN values because there wasn't a previous day's data to shift to that day.*

Next, let's shift the sales prices by five days.

In [ ]:
shifted_forward5 = data.shift(5)
shifted_forward5.head(10)

We can also use negative numbers to shift the sales values in the reverse direction.

In [ ]:
shifted_backward = data.shift(-1)
shifted_backward.head()

Lags can be used to calculate the changes in the values you are tracking with your time series data. In this case, we can use Pandas' `.shift()` method to look at the changes in sales. 

Let's create a new column in our Rossman DataFrame that contains the previous day's sales. 

*Note that we add `.copy()` to the end of the chained assignment to explicitly tell Pandas that this will be a copy and not a view. Here is a useful [video](https://www.youtube.com/watch?v=4R4WsDJ-KVc) that helps explain how to avoid SettingCopyWithWarning errors in Pandas.*

In [ ]:
data['Prev Day Sales'] = data['Sales'].shift(1).copy()
data.head()

Using our new column, it's simple to calculate the one-day change in sales at Store 1. Let's create a new column for this value in our DataFrame as well.

In [ ]:
data['Sales Change'] = data['Sales'] - data['Prev Day Sales'].copy()
data.head()

In [ ]:
data["Percentage Change"] = ((data['Sales'] / data["Prev Day Sales"]) * 100).copy()

In [ ]:
data.head()

## Time Series modeling with FBProphet
You will need to install prophet for this notebook. While we've used `pip install` in the past, it often causes errors. So, we'll use an alternative: `conda install -c conda-forge fbprophet`

In [ ]:
pip install fbprophet

In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt
 
%matplotlib inline
 
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

# Read in the data

Read the data in from the retail sales CSV file in the examples folder then set the index to the 'date' column. We are also parsing dates in the data file.

In [ ]:
#Read in the bikeshare data, and change the datetime column to a datetime object. DO NOT SET TO INDEX
bikes = pd.read_csv('./data/bikeshare.csv')

In [ ]:
#example: sales data
sales_df = pd.read_csv('./data/retail_sales.csv', parse_dates = True, low_memory = False)

In [ ]:
sales_df.head()

# Prepare for Prophet

For prophet to work, we need to change the names of these columns to 'ds' and 'y', so lets just create a new dataframe and keep our old one handy (you'll see why later). The new dataframe will initially be created with an integer index so we can rename the columns

In [ ]:
sales_df.rename(columns={'date':'ds', 'sales':'y'}, inplace = True)

In [ ]:
sales_df.head()

Now's a good time to take a look at your data.  Plot the data using pandas' ```plot``` function

In [ ]:
sales_df.set_index('ds').y.plot()

When working with time-series data, its good to take a look at the data to determine if trends exist, whether it is stationary, has any outliers and/or any other anamolies. Facebook prophet's example uses the log-transform as a way to remove some of these anomolies but it isn't the absolute 'best' way to do this...but given that its the example and a simple data series, I'll follow their lead for now.  Taking the log of a number is easily reversible to be able to see your original data. 

To log-transform your data, you can use numpy's log() function

In [ ]:
sales_df['y'] = np.log(sales_df['y'])

In [ ]:
sales_df.tail()

In [ ]:
sales_df.set_index('ds').y.plot()

As you can see in the above chart, the plot looks the same as the first one but just at a different scale.

## Now do the same with the bikeshare data


In [ ]:
bikes.rename(columns = {'datetime': 'ds', 'count': 'y'}, inplace = True)

In [ ]:
bikes['ds'] = pd.to_datetime(bikes.ds)

In [ ]:
bikes.set_index('ds').y.plot()

In [ ]:
bikes['y2'] = np.log(bikes['y'])

In [ ]:
bikes.set_index('ds').y2.plot()

In [ ]:
bike_2 = bikes.set_index('ds').resample('D').mean()

In [ ]:
bike_2.reset_index(inplace = True)
bike_2

## The math behind FBProphet

Prophet is a procedure for forecasting time series data based on an __additive__ model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects.

y(t) = g(t) + s(t) + h(t) + e(t)

g(t)

- trend models non-periodic changes (i.e. growth over time)

s(t)

- seasonality presents periodic changes (i.e. weekly, monthly, yearly)

h(t)

- ties in effects of holidays (on potentially irregular schedules ≥ 1 day(s))

e(t)

- covers idiosyncratic changes not accommodated by the model

The procedure provides two possible trend models for g(t), “a saturating growth model, and a piecewise linear model.”

- Linear Trend with Changepoints: For use when there is a 'maximum capacity', this is the default
- Saturating Growth Model: For use when there are no constants, and focus is on capacity for growth

# Running Prophet

Now, let's set prophet up to begin modeling our data.

Note: Since we are using monthly data, you'll see a message from Prophet saying ```Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.```  This is OK since we are workign with monthly data but you can disable it by using ```weekly_seasonality=True``` in the instantiation of Prophet.

In [ ]:
model_sales = Prophet()
model_sales.fit(sales_df)

Forecasting is fairly useless unless you can look into the future, so we need to add some future dates to our dataframe. For this example, I want to forecast 2 years into the future, so I'll built a future dataframe with 24 periods since we are working with monthly data. Note the ```freq='m'``` inclusion to ensure we are adding 24 months of data.

This can be done with the following code:


In [ ]:
future_sales = model_sales.make_future_dataframe(periods=24, freq = 'm')
future_sales.tail()

To forecast this future data, we need to run it through Prophet's model.

In [ ]:
forecast_sales = model_sales.predict(future)

The resulting forecast dataframe contains quite a bit of data, but we really only care about a few columns.  First, let's look at the full dataframe:

In [ ]:
forecast_sales.tail()

We really only want to look at yhat, yhat_lower and yhat_upper, so we can do that with:

In [ ]:
forecast_sales[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# Plotting Prophet results

Prophet has a plotting mechanism called ```plot```.  This plot functionality draws the original data (black dots), the model (blue line) and the error of the forecast (shaded blue area).

In [ ]:
model_sales.plot(forecast_sales)

## Evaluating Prophet
The shaded blue area is the error of the forecast. But we can only eyeball it. Let's look at the R-squared (amount of variance) and Mean Squared Error. 

In [ ]:
#To do this, we have to get the y-hat and original y's from the data
metric_df_sales = forecast_sales.set_index('ds')[['yhat']].join(sales_df.set_index('ds').y).reset_index()
metric_df_sales.tail()

In [ ]:
# The tail has NaN values, because they're predictions - there was no real Y. Let's drop those for model evaluation.
metric_df_sales.dropna(inplace = True)

In [ ]:
#Let's take a look at the numbers
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
print("R-squared: ", r2_score(metric_df_sales.y, metric_df_sales.yhat))
print("Mean Squared Error: ", mean_squared_error(metric_df_sales.y, metric_df_sales.yhat))
print("RMSE: ", np.sqrt(mean_squared_error(metric_df_sales.y, metric_df_sales.yhat)))


## Now try with bikeshare data

In [ ]:
model_bikes = Prophet()
model_bikes.fit(bikes[['ds', 'y']])

In [ ]:
future_bikes = model_bikes.make_future_dataframe(periods=365, freq = 'd')
future_bikes.tail()

In [ ]:
forecast_bikes = model_bikes.predict(future_bikes)
model.plot(forecast_bikes);

In [ ]:
metric_df_bikes = forecast_bikes.set_index('ds')[['yhat']].join(bikes.set_index('ds').y).reset_index()
metric_df_bikes.dropna(inplace = True)

print("R-squared: ", r2_score(metric_df_bikes.y, metric_df_bikes.yhat))
print("Mean Squared Error: ", mean_squared_error(metric_df_bikes.y, metric_df_bikes.yhat))
print("RMSE: ", np.sqrt(mean_squared_error(metric_df_bikes.y, metric_df_bikes.yhat)))

In [ ]:
bikes.y.describe()

In [ ]:
forecast

## Accounting for Seasonality and Trends

We can see from this data that there is a spike in the same month each year. While spike could be due to many different reasons, let's assume its because there's a major promotion that this company runs every year at that time, which is in December for this dataset.

When patterns repeat over *known, fixed periods* of time within a data set, we call this **seasonality**. A seasonal pattern exists when a series is influenced by factors related to the cyclic nature of time — i.e., time of month, quarter, year, etc. Seasonality is of a fixed and known period, otherwise it is not truly seasonality. Additionally, it must be either attributed to another factor or counted as a set of anomalous events in the data.

### Prophet calls them "holidays"

Because we know this promotion occurs every december, we want to use this knowledge to help prophet better forecast those months, so we'll use prohpet's ```holiday``` construct (explained here https://facebookincubator.github.io/prophet/docs/holiday_effects.html).

The holiday object is a pandas dataframe with the holiday and date of the holiday. For this example, the construct would look like this:

```promotions = pd.DataFrame({
  'holiday': 'december_promotion',
  'ds': pd.to_datetime(['2009-12-01', '2010-12-01', '2011-12-01', '2012-12-01',
                        '2013-12-01', '2014-12-01', '2015-12-01']),
  'lower_window': 0,
  'upper_window': 0,
})```

This ```promotions``` dataframe consisists of promotion dates for Dec in 2009 through 2015,  The ```lower_window``` and ```upper_window``` values are set to zero to indicate that we don't want prophet to consider any other months than the ones listed.

In [ ]:
# Build the promotions dataframe from above here - be sure you understand the syntax and logic!
promotions = pd.DataFrame({
  'holiday': 'december_promotion',
  'ds': pd.to_datetime(['2009-12-01', '2010-12-01', '2011-12-01', '2012-12-01',
                        '2013-12-01', '2014-12-01', '2015-12-01']),
  'lower_window': 0,
  'upper_window': 0,
})

In [ ]:
promotions.head()

In [ ]:
# Remember, we need to log-transform for prophet
sales_df['y'] = np.log(sales_df['y'])
sales_df.head()

In [ ]:
#Now let's set up prophet to model our data using holidays
model_sales = Prophet(holidays=promotions)
model_sales.fit(sales_df)

In [ ]:
#We've instantiated the model, so now we need to build our future dates to forecast into!
future_sales = model_sales.make_future_dataframe(periods=24, freq = 'm')
future_sales.tail()

#... and then run our future data through prophet's model
forecast_sales = model_sales.predict(future_sales)

forecast_sales.head()

In [ ]:
#while our new df contains a bit of data, we only care about a few features...
forecast_sales[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


## Visualizing with holidays!
Same as above at first blush!

In [ ]:
#use Prophet's .plot() method to visualize your timeseries.
model_sales.plot(forecast_sales);

Prophet also allows you to examine the ```components``` of a timeseries using the ```.plot_components()``` method

In [ ]:
model_sales.plot_components(forecast_sales);

## Why holidays matter
Let's re-run our prophet model without holidays, for comparison

In [ ]:
model_no_holiday = Prophet()
model_no_holiday.fit(sales_df)

In [ ]:
future_no_holiday = model_no_holiday.make_future_dataframe(periods=24, freq = 'm')
future_no_holiday.tail()

In [ ]:
forecast_no_holiday = model_no_holiday.predict(future_sales)

There probably won't be a massive difference, given the small amount of data with which we're working on this example. But with greater data comes greater variance...

In [ ]:
#join the dataframes
forecast_sales.set_index('ds', inplace=True)
forecast_no_holiday.set_index('ds', inplace=True)
compared_df = forecast_sales.join(forecast_no_holiday, rsuffix="_no_holiday")

In [ ]:
#we're only interested in the predictions, and let's move back to the original scale
compared_df= np.exp(compared_df[['yhat', 'yhat_no_holiday']])

In [ ]:
# Create a feature that is the percentage difference between holiday vs. none
compared_df['diff_per'] = 100 * (compared_df['yhat'] - compared_df['yhat_no_holiday']) / compared_df['yhat_no_holiday']
print("difference: ", round(compared_df.diff_per.mean(), 2), "%")

While the difference here is less than 10%, that can be a large amount of money left on the table if your business is a global enterprise!

In [ ]:
#To do this, we have to get the y-hat and original y's from the data
metric_df_sales_no_holiday = forecast_no_holiday[['yhat']].join(sales_df.set_index('ds').y).reset_index()
metric_df_sales_no_holiday.tail()

In [ ]:
# The tail has NaN values, because they're predictions - there was no real Y. Let's drop those for model evaluation.
metric_df_sales_no_holiday.dropna(inplace = True)

In [ ]:
print("R-squared: ", r2_score(metric_df_sales_no_holiday.y, metric_df_sales_no_holiday.yhat))
print("Mean Squared Error: ", mean_squared_error(metric_df_sales_no_holiday.y, metric_df_sales_no_holiday.yhat))
print("RMSE: ", np.sqrt(mean_squared_error(metric_df_sales_no_holiday.y, metric_df_sales_no_holiday.yhat)))

## Adding Regressors

### The general recipe

m = Prophet()

m.add_regressor('add1')

m.add_regressor('add2')

...

m.fit(df)

future = m.make_future_dataframe(periods=10)

forecast = m.predict(future)

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(15)



*Note that the additional variables should have values for your future (test) data. If you don't have them, you could start by using FBProphet to predict add1 and add2 with  with univariate timeseries, and then predict y with add_regressor and the predicted add1 and add2 as future values of the additional variables.*



In [ ]:
temperature = bikes[['ds', 'temp']]
temperature.rename(columns = {'temp': 'y'}, inplace = True)

In [ ]:
temp_m = Prophet()
temp_m.fit(temperature[['ds', 'y']])
temp_m_future = temp_m.make_future_dataframe(periods = 365, freq = 'D')
temp_m_forecast = temp_m.predict(future)

In [ ]:
yhat_temp = temp_m_forecast['yhat']

In [ ]:
m = Prophet()
m.add_regressor('temp')
m.fit(bikes[['ds', 'y', 'temp', 'humidity']])

In [ ]:
future = m.make_future_dataframe(periods = 365, freq = 'D')
future['temp'] = yhat_temp
forecast = m.predict(future)

In [ ]:
#Another approach to evaluation
from fbprophet.diagnostics import cross_validation, performance_metrics
df_cv = cross_validation(m, horizon='30 days')
df_p = performance_metrics(df_cv)
df_p.tail(5)

In [ ]:
#How did your model do? Does adding additional regressors help or inhibit your model? 
metric_df_bikes = forecast.set_index('ds')[['yhat']].join(bikes.set_index('ds').y).reset_index()
metric_df_bikes.dropna(inplace = True)

print("R-squared: ", r2_score(metric_df_bikes.y, metric_df_bikes.yhat))
print("Mean Squared Error: ", mean_squared_error(metric_df_bikes.y, metric_df_bikes.yhat))
print("RMSE: ", np.sqrt(mean_squared_error(metric_df_bikes.y, metric_df_bikes.yhat)))

# Prophet for Market prediction - lab time!
Prophet can detect changepoints in timeseries data, and we can often use it to our advantage. Let's grab FRED economic data and see how this goes.

In [ ]:
#Download 01/2009 - current S&P500 data at https://fred.stlouisfed.org/series/SP500 and import it into pandas
market_df = 

In [ ]:
#Now prepare your data for prophet. Hint: prophet needs "ds" and a log-transformed "y" to work


In [ ]:
#Instantiate the model, and fit our data
model = Prophet()


In [ ]:
#build the future dataframe, forecasting for 1 year from now. THen create a forecast by passing the future into model.predict()
future = model.make_future_dataframe(periods = 365)
forecast = 

In [ ]:
#now plot it!


It's hard to see what's going on here, in part because we have such a condensed visual. Let's look at the last 2 years of forecast vs. actual without looking at the future - we are just interested ing etting a visual of theerror between actual vs. forecast

In [ ]:
#Start by setting the index of the forecast df to the ds, and joining it to the marekt_df
two_years = 

#now set two_years equal to the last 800 data points
two_years = two_years[['SP500', 'yhat', 'yhat_upper', 'yhat_lower' ]].dropna().tail(800)

In [ ]:
#now trasnform the predictions back to the original scale of the data
two_years['yhat']=np.exp(two_years.yhat)
two_years['yhat_upper']=np.exp(two_years.yhat_upper)
two_years['yhat_lower']=np.exp(two_years.yhat_lower)

In [ ]:
#let's visualize the relationship between SP500 and yhat (our prediction) using pandas .plot()


Our forecast does great at trending, but doesn't do well at catching the volatility of the market. This would be very good for 'riding trends', but not so good for catching peaks and dips. 

We can see this in the numbers as well

In [ ]:
#calculate the r2


In [ ]:
#MAE


In [ ]:
#RMSE


Another way to tlook at the usefulness of this forecast is to plot the upper and lower confidence bands against the actuals. We can do that with a plot that combines yhat_upper and yhat_lower with the rest into a matplotlib subplot. 

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(two_years.SP500)
ax1.plot(two_years.yhat)
ax1.plot(two_years.yhat_upper, color='black',  linestyle=':', alpha=0.5)
ax1.plot(two_years.yhat_lower, color='black',  linestyle=':', alpha=0.5)

ax1.set_title('Actual S&P 500 (Orange) vs S&P 500 Forecasted Upper & Lower Confidence (Black)')
ax1.set_ylabel('Price')
ax1.set_xlabel('Date')

As we saw above, if you're trying to do shortterm trading then this model is useless. But if you are investing with a timeframe of months to years, this forecast might provide some value. 

In [ ]:
#We can also look at this from the full df. Here I build it manually again!
fig, ax1 = plt.subplots()
ax1.plot(full_df.SP500)
ax1.plot(full_df.yhat, color='black', linestyle=':')
ax1.fill_between(full_df.index, np.exp(full_df['yhat_upper']), np.exp(full_df['yhat_lower']), alpha=0.5, color='darkgray')
ax1.set_title('Actual S&P 500 (Orange) vs S&P 500 Forecasted (Black) with Confidence Bands')
ax1.set_ylabel('Price')
ax1.set_xlabel('Date')

L=ax1.legend() #get the legend
L.get_texts()[0].set_text('S&P 500 Actual') #change the legend text for 1st plot
L.get_texts()[1].set_text('S&P 5600 Forecasted') #change the legend text for 2nd plot